## Summary

Repo: https://github.com/pgahq/instructor-groq-openai-llm-examples

This notebook shows how to use Instructor with nested models to make an analysis (of a new business idea). At the end, it'll even suggest a new, improved version of the business idea.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.



In [2]:
!pip install --quiet instructor groq openai jsonref

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [3]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field
from typing import Optional, List
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass


In [7]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())


company = "Luigi's Murano Glass Company"
company_description = """
Luigi's Murano Glass Company is a leading manufacturer of high-quality, affordable, and stylish glass products. With a focus on innovative design and durability, Luigi's Murano Glass Company offers a wide range of glass solutions to meet the needs of both residential and commercial customers. With over 20 years of manufacturing experience, Luigi's Murano Glass Company has a proven track record of delivering high-quality glass products at unbeatable prices. Our factory store is located in Murano, Italy.
"""

new_business_idea = """
New Business Idea: Partnering with US Interior Architecture Firms to Distribute Official Murano Glass Sinks

Objectives:
1. Establish Strategic Partnerships:
   - Collaborate with top US interior architecture firms to integrate our Murano glass sinks into their design projects.
   - Develop co-branded marketing materials to highlight the partnership and the unique value of Murano glass sinks.
   - Offer exclusive deals and discounts to partner firms to incentivize the use of our products in their projects.

2. Increase Market Penetration:
   - Expand our market reach by tapping into the client base of established interior architecture firms.
   - Utilize the firms' existing relationships with high-end residential and commercial clients to introduce our products.
   - Leverage the firms' expertise to tailor our product offerings to meet the specific needs and preferences of their clients.

3. Enhance Brand Visibility:
   - Elevate the brand visibility of Luigi's Murano Glass Company through high-profile design projects.
   - Feature our Murano glass sinks in prominent design publications and online platforms.
   - Participate in industry events and trade shows alongside our partner firms to showcase our products.

4. Drive Revenue Growth:
   - Generate additional revenue through increased sales of Murano glass sinks to architecture firms and their clients.
   - Offer premium pricing for custom-designed sinks that cater to the unique requirements of high-end projects.
   - Explore opportunities for bulk sales and long-term contracts with partner firms.

Execution Plan:
1. Identify Potential Partners:
   - Research and shortlist top interior architecture firms in key US markets.
   - Reach out to potential partners with a compelling proposal highlighting the benefits of collaboration.

2. Develop Partnership Agreements:
   - Draft and negotiate partnership agreements that outline the terms of collaboration, including pricing, exclusivity, and marketing commitments.
   - Ensure that the agreements align with our business objectives and provide mutual benefits.

3. Create Marketing and Sales Materials:
   - Develop co-branded marketing materials, including brochures, catalogs, and online content, to promote the partnership.
   - Train our sales team to effectively communicate the value proposition of our Murano glass sinks to architecture firms and their clients.

4. Launch and Promote the Partnership:
   - Announce the partnership through press releases, social media, and industry publications.
   - Host launch events and webinars to introduce the partnership to the design community and potential clients.

5. Monitor and Evaluate Performance:
   - Track the performance of the partnership through sales data, client feedback, and market analysis.
   - Regularly review and adjust the partnership strategy to ensure it meets our business objectives and maximizes value for both parties.

Market Analysis:
- Market Need Score: Highlight the growing demand for unique and high-quality design elements in interior architecture.
- Market Size Score: Emphasize the substantial market size of the US interior architecture industry and the potential for growth.
- Competitive Landscape Score: Address the competitive landscape and the need for differentiation through exclusive partnerships and unique product offerings.

Overall Strategy:
- Ensure a well-rounded and promising strategy with significant potential benefits through strategic partnerships.
- Address challenges such as competitive landscape, partnership management, and market adaptation.
- Focus on enhancing brand visibility, increasing market penetration, and driving revenue growth through collaboration with interior architecture firms.
- Develop contingency plans to mitigate potential risks and ensure smooth execution of the partnership strategy.

"""

In [9]:

class Score(BaseModel):
    score: float = Field(description=f"""
        **Score**
        
        Score for the strategy evaluation.
    """)
    explanation: str = Field(description=f"""
        **Explanation**
        
        Explanation / justification for the score.
    """)

class Objective(BaseModel):
    name: str = Field(description=f"""
        **Objective Name**
        
        Name of the strategic objective derived from the strategy document.
    """)
    description: str = Field(description=f"""
        **Objective Description**
        
        Description of the strategic objective derived from the strategy document.
    """)
    relevance_to_objective_score: Score = Field(description=f"""
        **Relevance to Objective Score (1-5 integer)**
        
        Rate the relevance of this objective on a 5-point scale, from 1 (least relevant) to 5 (most relevant).

        **Justification for Relevance to Objective Score**
        
        Provide a detailed explanation justifying the assigned relevance score, including its alignment with the company's mission and goals.
        
        - **Impact on Clients**: How does this objective serve the clients?
        - **Growth of the Business**: In what way does it contribute to the growth of the business?
        - **Alignment with Core Values**: How does it align with the company's core values?
    """)

class MarketAnalysis(BaseModel):
    market_need_score: Score = Field(description=f"""
        **Market Need Score (1-5 integer)**
        
        Rate the market need for the business idea on a 5-point scale, from 1 (low need) to 5 (high need).

        **Justification for Market Need Score**
        
        Provide a detailed explanation justifying the market need score, including the pain points and needs addressed by the business idea.
        
        - **Assessment of Demand**: What is the evidence of demand for this idea?
        - **Client Feedback**: Have clients expressed a need for this?
    """)
    market_size_score: Score = Field(description=f"""
        **Market Size Score (1-5 integer)**
        
        Rate the market size on a 5-point scale, from 1 (small market) to 5 (large market).

        **Justification for Market Size Score**
        
        Provide a detailed explanation justifying the market size score, including market size estimates and growth potential.
        
        - **Market Research**: What does market research indicate about the size and potential growth?
        - **Competitive Landscape**: How crowded is the market?
    """)
    competitive_landscape_score: Score = Field(description=f"""
        **Competitive Landscape Score (1-5 integer)**
        
        Rate the competitive landscape on a 5-point scale, from 1 (high competition) to 5 (low competition).

        **Justification for Competitive Landscape Score**
        
        Provide a detailed explanation justifying the competitive landscape score, including key competitors and market gaps.
        
        - **Competitive Dynamics**: Who are the key players and how competitive is the sector?
        - **Differentiation**: How does this business idea stand out from competitors?
    """)

class FinancialAnalysis(BaseModel):
    revenue_projection_score: Score = Field(description=f"""
        **Revenue Projection Score (1-5 integer)**
        
        Rate the revenue projection on a 5-point scale, from 1 (low projection) to 5 (high projection).

        **Justification for Revenue Projection Score**
        
        Provide a detailed explanation justifying the revenue projection score, including potential revenue streams and growth expectations.
        
        - **Revenue Channels**: What are the main sources of revenue from this idea?
        - **Growth Trajectory**: What are the short-term and long-term revenue projections?
    """)
    cost_analysis_score: Score = Field(description=f"""
        **Cost Analysis Score (1-5 integer)**
        
        Rate the cost analysis on a 5-point scale, from 1 (high costs) to 5 (low costs).

        **Justification for Cost Analysis Score**
        
        Provide a detailed explanation justifying the cost analysis score, including initial investments, operational costs, and ongoing expenses.
        
        - **Start-up Costs**: What are the initial investments required?
        - **Operational Efficiency**: How efficient is the cost structure?
    """)
    profitability_score: Score = Field(description=f"""
        **Profitability Score (1-5 integer)**
        
        Rate the profitability on a 5-point scale, from 1 (low profitability) to 5 (high profitability).

        **Justification for Profitability Score**
        
        Provide a detailed explanation justifying the profitability score, including potential margins and return on investment.
        
        - **Margin Analysis**: What are the expected profit margins?
        - **Return on Investment**: How quickly can we expect to break even and see returns?
    """)
    funding_requirements_score: Score = Field(description=f"""
        **Funding Requirements Score (1-5 integer)**
        
        Rate the funding requirements on a 5-point scale, from 1 (high funding needs) to 5 (low funding needs).

        **Justification for Funding Requirements Score**
        
        Provide a detailed explanation justifying the funding requirements score, including potential funding sources and financial sustainability.
        
        - **Capital Needs**: How much capital is required and for what?
        - **Funding Sources**: What are potential sources of funding, both internal and external?
    """)

class OperationalFeasibility(BaseModel):
    resource_availability_score: Score = Field(description=f"""
        **Resource Availability Score (1-5 integer)**
        
        Rate the availability of necessary resources on a 5-point scale, from 1 (low availability) to 5 (high availability).

        **Justification for Resource Availability Score**
        
        Provide a detailed explanation justifying the resource availability score, including human, technological, and financial resources.
        
        - **Human Resources**: Are the required skills and personnel available?
        - **Technological Resources**: Are the necessary technologies in place and accessible?
    """)
    operational_requirements_score: Score = Field(description=f"""
        **Operational Requirements Score (1-5 integer)**
        
        Rate the operational requirements on a 5-point scale, from 1 (high complexity) to 5 (low complexity).

        **Justification for Operational Requirements Score**
        
        Provide a detailed explanation justifying the operational requirements score, including the complexity and feasibility of operational processes.
        
        - **Operational Complexity**: What operational processes are required and how complex are they?
        - **Feasibility**: How feasible is it to implement these processes given current capabilities?
    """)
    scalability_score: Score = Field(description=f"""
        **Scalability Score (1-5 integer)**
        
        Rate the scalability on a 5-point scale, from 1 (low scalability) to 5 (high scalability).

        **Justification for Scalability Score**
        
        Provide a detailed explanation justifying the scalability score, including the potential to scale the business and handle growth.
        
        - **Scalability Potential**: How scalable is the business model?
        - **Growth Capacity**: What infrastructure is needed to support growth?
    """)

class RiskAnalysis(BaseModel):
    market_risk_score: Score = Field(description=f"""
        **Market Risk Score (1-5 integer)**
        
        Rate the market risk on a 5-point scale, from 1 (high risk) to 5 (low risk).

        **Justification for Market Risk Score**
        
        Provide a detailed explanation justifying the market risk score, including external risks such as market fluctuations and competition.
        
        - **Market Volatility**: What is the level of market fluctuation risk?
        - **Competitive Risk**: How does competition pose a risk?
    """)
    operational_risk_score: Score = Field(description=f"""
        **Operational Risk Score (1-5 integer)**
        
        Rate the operational risk on a 5-point scale, from 1 (high risk) to 5 (low risk).

        **Justification for Operational Risk Score**
        
        Provide a detailed explanation justifying the operational risk score, including internal risks such as resource constraints and execution challenges.
        
        - **Execution Risk**: What are the risks associated with executing the operational plans?
        - **Resource Risk**: How likely are resource shortages or constraints?
    """)
    financial_risk_score: Score = Field(description=f"""
        **Financial Risk Score (1-5 integer)**
        
        Rate the financial risk on a 5-point scale, from 1 (high risk) to 5 (low risk).

        **Justification for Financial Risk Score**
        
        Provide a detailed explanation justifying the financial risk score, including financial risks such as funding and revenue variability.
        
        - **Funding Risk**: What are the risks associated with securing necessary funding?
        - **Revenue Risk**: How stable are the anticipated revenue streams?
    """)
    legal_regulatory_risk_score: Score = Field(description=f"""
        **Legal and Regulatory Risk Score (1-5 integer)**
        
        Rate the legal and regulatory risk on a 5-point scale, from 1 (high risk) to 5 (low risk).

        **Justification for Legal and Regulatory Risk Score** 
        
        Provide a detailed explanation justifying the legal and regulatory risk score, including compliance with relevant laws and regulations.
        
        - **Compliance**: What are the legal compliance requirements and risks?
        - **Regulatory Changes**: How susceptible is the business to changes in regulations?
    """)

class ImpactAnalysis(BaseModel):
    client_impact_score: Score = Field(description=f"""
        **Client Impact Score (1-5 integer)**
        
        Rate the impact on clients on a 5-point scale, from 1 (low impact) to 5 (high impact).

        **Justification for Client Impact Score**
        
        Provide a detailed explanation justifying the client impact score, including how the business idea benefits the clients.
        
        - **Client Services**: How does this idea improve services for clients?
        - **Client Satisfaction**: What is the potential impact on client satisfaction and engagement?
    """)
    business_growth_score: Score = Field(description=f"""
        **Business Growth Impact Score (1-5 integer)**
        
        Rate the impact on the growth of the business on a 5-point scale, from 1 (low impact) to 5 (high impact).

        **Justification for Business Growth Impact Score**
        
        Provide a detailed explanation justifying the business growth impact score, including how the business idea contributes to the growth of the company.
        
        - **Market Expansion**: How will this idea drive market expansion?
        - **Outreach and Engagement**: How does it help in reaching new demographics or expanding current ones?
    """)
    brand_impact_score: Score = Field(description=f"""
        **Brand Impact Score (1-5 integer)**

        Rate the impact on the company's brand on a 5-point scale, from 1 (low impact) to 5 (high impact).

        **Justification for Brand Impact Score**
        
        Provide a detailed explanation justifying the brand impact score, including potential impacts on the company's brand and reputation.
        
        - **Brand Equity**: How does this idea enhance or maintain brand equity?
        - **Public Perception**: What are the potential effects on public perception of the company?
    """)
    sustainability_score: Score = Field(description=f"""
        **Sustainability Impact Score (1-5 integer)**
        
        Rate the sustainability impact on a 5-point scale, from 1 (low impact) to 5 (high impact).

        **Justification for Sustainability Impact Score**
        
        Provide a detailed explanation justifying the sustainability impact score, including contributions to environmental and social sustainability goals.
        
        - **Environmental Impact**: Does this idea support environmental sustainability initiatives?
        - **Social Responsibility**: What are the social implications and benefits?
    """)

class StrategyEvaluation(BaseModel):
    """
    Critical analysis of a new business idea.
    """
    title: str = Field(description=f"""
        **Strategy Document Title** 
        
        Title of the strategy document being evaluated.
    """)
    objectives: List[Objective] = Field(description=f"""
        **Objectives and Relevance Scores** 
        
        List of strategic objectives and their respective relevance scores and justifications.
    """)
    market_analysis: MarketAnalysis = Field(description=f"""
        **Market Analysis Scores and Justifications** 
        
        Market analysis scores and detailed justifications, including market need, market size, and competitive landscape.
    """)
    financial_analysis: FinancialAnalysis = Field(description=f"""
        **Financial Analysis Scores and Justifications** 
        
        Financial analysis scores and detailed justifications, including revenue projections, cost analysis, profitability, and funding requirements.
    """)
    operational_feasibility: OperationalFeasibility = Field(description=f"""
        **Operational Feasibility Scores and Justifications** 
        
        Operational feasibility scores and detailed justifications, including resource availability, operational requirements, and scalability.
    """)
    risk_analysis: RiskAnalysis = Field(description=f"""
        **Risk Analysis Scores and Justifications** 
        
        Risk analysis scores and detailed justifications, including market risk, operational risk, financial risk, and legal/regulatory risk.
    """)
    impact_analysis: ImpactAnalysis = Field(description=f"""
        **Impact Analysis Scores and Justifications** 
        
        Impact analysis scores and detailed justifications, including client impact, business growth impact, and brand impact.
    """)
    all_scores: List[Score] = Field(description=f"""
        **All Scores**
        
        List of all scores, including strategic objectives, market analysis, financial analysis, operational feasibility, risk analysis, and impact analysis.
    """)
    overall_score: float = Field(description=f"""
        **Overall Score**
        
        Overall score (an average of all scores, rounded to one decimal place), including strategic objectives, market analysis, financial analysis, operational feasibility, risk analysis, and impact analysis.
    """)
    overall_scores_narrative: str = Field(description=f"""
        **Overall Scores Narrative**
        
        Narrative summary of most significant drivers / factors for the overall score.
    """)
    improved_new_business_idea_text: str = Field(description=f"""
        **Improved New Business Idea Text**
        
        Complete revised proposal text to maximize the overall score. The only thing that needs to stay the same is the title. Use markdown. No commentary or other text...just the revised business idea text.
    """)


## Note: running this may take about a minute!

In [10]:

result = client.chat.completions.create(
    model="llama3-70b-8192" if inference_provider == "groq" else "gpt-4o",
    response_model=StrategyEvaluation,
    temperature=0.0,
    messages=[
      {"role": "user", "content": "Company: " + company},
      {"role": "user", "content": "Company Description: " + company_description},
      {"role": "user", "content": "New Business Idea: " + new_business_idea}
      ]
    )

print(result.model_dump_json(indent=4))

{
    "title": "Partnering with US Interior Architecture Firms to Distribute Official Murano Glass Sinks",
    "objectives": [
        {
            "name": "Establish Strategic Partnerships",
            "description": "Collaborate with top US interior architecture firms to integrate our Murano glass sinks into their design projects. Develop co-branded marketing materials to highlight the partnership and the unique value of Murano glass sinks. Offer exclusive deals and discounts to partner firms to incentivize the use of our products in their projects.",
            "relevance_to_objective_score": {
                "score": 5.0,
                "explanation": "This objective is highly relevant as it directly aligns with the company's mission to expand its market reach and enhance brand visibility. By partnering with top US interior architecture firms, Luigi's Murano Glass Company can leverage their expertise and client base to introduce its products to a wider audience. This collabora